# Lagged Series
This creates a Pandas DataFrame that stores the percentage returns of the adjusted closing value of a stock obtained from AlphaVantage, along with a
number of lagged returns from the prior trading days (lags defaults to 5 days). Trading volume, as well as the Direction from the previous day, are also included.



In [32]:
from datetime import datetime as dt, timedelta as td
import os
import sys
import numpy as np
import pandas as pd
import sklearn

def create_lagged_series(df,lags=[5,10,20,60]):
    """
    Parameters
    ----------
    df= time series data frame inluding adj_close_price and vol
    symbol : ‘str‘
    lags : int list, optional The number of days to ’lag’ the series by

    """ 
    
    # Clear any NAs
    df=df.dropna()
    
    
    # Creating the new lagged DataFrame
    tslag = pd.DataFrame(index=df.index)
    tslag['today'] = df['adj_close_price']
    tslag['volume'] = df['volume']
    
    # Create returns DataFrame
    tsret = pd.DataFrame(index=tslag.index)
    tsret['volume'] = tslag['volume']
    tsret['today'] = tslag['today'].pct_change()
    
    # Creating the shifted lag series of prior trading period close values
    for i in lags:
        tsret['Lag%s' % str(i)] = df['adj_close_price'].pct_change().rolling(i).mean().shift(1)
    
        
    # Create the "Direction" column (+1 or -1) indicating an up/down day
    tsret['Direction'] = np.sign(tsret['today'])
    
    tsret=tsret.dropna()
        
    return tsret